In [ ]:
from sklearn import svm
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pops = ["V1L4E", "V1L4I", "V1L23E", "V1L23I", "V2L4E", "V2L4I", "V2L23E", "V2L23I"]
sets = ["MNIST_train","MNIST_test","CIFAR-10_test_gray","CIFAR-10_train_gray","SVHN_test","SVHN_train", "EMNIST_test","EMNIST_train","ETH80_gray_vector","CalTec101_subset"]#CalTec101_images,STL10_train,STL10_test
#sets = ["MNIST_train","MNIST_test","CIFAR-10_test_gray","CIFAR-10_train_gray","SVHN_test","SVHN_train", "ETH80_gray_vector","CalTec101_subset"]#CalTec101_images,STL10_train,STL10_test
noImgs = [60000,10000,10000,50000,26032,73257,18800,112800,3280,1233]
#conditions = ["30on", "30off", "42on", "42off", "100on", "100off"]
times = ["30", "100", "42"]
feedback = ["on","off"]
vars = ["trace","average"]
accuracies = pd.DataFrame(columns=["Dataset","Recording Time", "Feedback", "Population", "Accuracy", "Variation"])

In [ ]:
evPath = "../RESULTS/variations/"+vars[0]+"eval/"+feedback[0]+"/"

In [ ]:
ds = []
for i,s in enumerate(sets):
    ds.append(np.concatenate([np.loadtxt("../DATA/"+s+"/"+str(k)) for k in range(noImgs[i])]).flatten())

In [ ]:
len(ds)

In [ ]:
fig, axs = plt.subplots(5,2, figsize = (10,15))
for i,s in enumerate(ds):
    sl = len(s)
    tmp = s[s!=0]
    c, b, _ = axs.flatten()[i].hist(tmp, 100)
    axs.flatten()[i].bar(tmp.mean(), c.max(), width=b[1]*.8, color = "red")
    axs.flatten()[i].text(.9, .9, 100*len(tmp)/sl)

# Evaluation

## Cut Datasets

## Housekeeping

In [ ]:
for v in vars:
    path = "../RESULTS/variations/"+v+"/"
    acc = pd.concat([pd.read_csv(path+"accuracieson.csv", index_col=0), pd.read_csv(path+"accuraciesoff.csv", index_col=0)])
    acc.drop_duplicates().to_csv(path+"accuracies.csv")

In [ ]:
path = "../RESULTS/variations/"+vars[0]+"/"
acc = pd.read_csv(path+"accuracies.csv", index_col=0)

In [ ]:
acc["trace"]

# Analysis

In [ ]:
dss = ['MNIST', 'CIFAR-10', 'SVHN', 'EMNIST', 'ETH80', 'CalTec101']
chL = [10., ]
acc = {v:pd.read_csv("../RESULTS/variations/"+v+"/accuracies.csv", index_col=0) for v in vars}

## Visualisation

In [ ]:
fig, axs = plt.subplots(6,4, figsize=(15,10))
for i,s in enumerate(dss):
    chL = 1
    for k,v in enumerate(vars):
        # Prep data
        tmp = acc[v]
        tmp = tmp[tmp["Dataset"]==s].reset_index()
        
        tmp = tmp[["Recording Time","Accuracy","Population","Feedback"]].pivot(index="Recording Time", values="Accuracy", columns=["Feedback","Population"])

        # On condition
        tmp["on"].reset_index(drop=True).plot(ax=axs[i][0+2*k], legend=False, xlabel="", style="o-", ylim=(0,100))
        axs[i][0+2*k].set_xticks([0,1,2],tmp["on"].index.values)
        # Off condition
        tmp["off"].reset_index(drop=True).plot(ax=axs[i][1+2*k], xlabel="", legend=False, style="o-", ylim=(0,100))
        axs[i][1+2*k].set_xticks([0,1,2],tmp["on"].index.values)
    axs[i][0].text(-.4, .5,s, transform = axs[i][0].transAxes)
    axs[i][3].legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=7)
    
axs[0][0].set_title("Feedback On")
axs[0][1].set_title("Feedback Off")
axs[0][2].set_title("Feedback On")
axs[0][3].set_title("Feedback Off")
fig.suptitle("Trace                                                                                                                            Average")
fig.tight_layout()

In [ ]:
s = dss[1]
chL = 0
vis = []
for v in vars:
    tmp = acc[v][acc[v]["Dataset"]==s]
    chL = tmp["Chance Level"].values[0]    
    tmp = tmp.sort_values("Feedback")
    tmp = tmp.pivot(index="Population", columns=["Feedback", "Recording Time"], values="Accuracy").round(2)
    tmp = tmp.reindex(columns=sorted(tmp.columns))
    vis.append(tmp.loc[pops])
print(s)
print(chL)
pd.concat(vis, axis=1, keys=vars)

## Condition Effects

In [ ]:
print("trace condition")
tmp = acc["trace"].drop("Chance Level", axis=1).sort_values(["Feedback","Dataset","Recording Time","Population"])
fTrace = tmp["Accuracy"].values

# Feedback Effect
print("  Feedback effect")
on = tmp[tmp["Feedback"]=="on"]["Accuracy"].values
off = tmp[tmp["Feedback"]=="off"]["Accuracy"].values
print(sp.stats.ttest_rel(on, off, alternative="less"))
# Time Effect
t30 = tmp[tmp["Recording Time"]==30]["Accuracy"].values
t42 = tmp[tmp["Recording Time"]==42]["Accuracy"].values
t100 = tmp[tmp["Recording Time"]==100]["Accuracy"].values
print("    t30 > t42")
print(sp.stats.ttest_rel(t30, t42, alternative="greater"))
print("    t42 > t100")
print(sp.stats.ttest_rel(t42, t100, alternative="greater"))


print("average condition")
tmp = acc["average"].drop("Chance Level", axis=1).sort_values(["Feedback","Dataset","Recording Time","Population"])
fAvg = tmp["Accuracy"].values
# Feedback Effect
print("  Feedback effect")
on = tmp[tmp["Feedback"]=="on"]["Accuracy"].values
off = tmp[tmp["Feedback"]=="off"]["Accuracy"].values
print(sp.stats.ttest_rel(on, off, alternative="two-sided"))
# Time Effect
t30 = tmp[tmp["Recording Time"]==30]["Accuracy"].values
t42 = tmp[tmp["Recording Time"]==42]["Accuracy"].values
t100 = tmp[tmp["Recording Time"]==100]["Accuracy"].values
print("    t30 > t42")
print(sp.stats.ttest_rel(t30, t42, alternative="greater"))
print("    t42 > t100")
print(sp.stats.ttest_rel(t42, t100, alternative="greater"))
print("    t30 > t100")
print(sp.stats.ttest_rel(t30, t100, alternative="greater"))

print("Design Differences: trace greater than average")
print(sp.stats.ttest_rel(fTrace, fAvg, alternative="greater"))

In [ ]:
print("trace condition")
tmp = acc["trace"].drop("Chance Level", axis=1).sort_values(["Feedback","Dataset","Recording Time","Population"])
tmp = tmp[tmp["Dataset"]!="ETH80"]
tmp = tmp[tmp["Dataset"]!="CalTec101"]
fTrace = tmp["Accuracy"].values

# Feedback Effect
print("  Feedback effect")
on = tmp[tmp["Feedback"]=="on"]["Accuracy"].values
off = tmp[tmp["Feedback"]=="off"]["Accuracy"].values
print(sp.stats.ttest_rel(on, off, alternative="less"))
# Time Effect
t30 = tmp[tmp["Recording Time"]==30]["Accuracy"].values
t42 = tmp[tmp["Recording Time"]==42]["Accuracy"].values
t100 = tmp[tmp["Recording Time"]==100]["Accuracy"].values
print("    t30 > t42")
print(sp.stats.ttest_rel(t30, t42, alternative="greater"))
print("    t42 > t100")
print(sp.stats.ttest_rel(t42, t100, alternative="greater"))


print("average condition")
tmp = acc["average"].drop("Chance Level", axis=1).sort_values(["Feedback","Dataset","Recording Time","Population"])
tmp = tmp[tmp["Dataset"]!="ETH80"]
tmp = tmp[tmp["Dataset"]!="CalTec101"]
fAvg = tmp["Accuracy"].values
# Feedback Effect
print("  Feedback effect")
on = tmp[tmp["Feedback"]=="on"]["Accuracy"].values
off = tmp[tmp["Feedback"]=="off"]["Accuracy"].values
print(sp.stats.ttest_rel(on, off, alternative="two-sided"))
# Time Effect
t30 = tmp[tmp["Recording Time"]==30]["Accuracy"].values
t42 = tmp[tmp["Recording Time"]==42]["Accuracy"].values
t100 = tmp[tmp["Recording Time"]==100]["Accuracy"].values
print("    t30 > t42")
print(sp.stats.ttest_rel(t30, t42, alternative="greater"))
print("    t42 > t100")
print(sp.stats.ttest_rel(t42, t100, alternative="greater"))

print("Design Differences: trace greater than average")
print(sp.stats.ttest_rel(fTrace, fAvg, alternative="greater"))

In [ ]:
print("Design Differences: trace greater than average")
print(sp.stats.ttest_rel(fTrace, fAvg, alternative="greater"))



In [ ]:
from sklearn.linear_model import LinearRegression as lr
from sklearn.preprocessing import PolynomialFeatures as pf

In [ ]:
for v in vars:
    np.set_printoptions(precision=2)
    tmp = acc[v]
    tmp["Feedback"] = tmp["Feedback"].replace(["on","off"], [1,0])
    tmp["Population"] = tmp["Population"].replace(pops, [0,0,1,1,2,2,3,3])
    tmp = tmp[tmp["Dataset"]!="ETH80"]
    tmp = tmp[tmp["Dataset"]!="CalTec101"]
    y = tmp["Accuracy"]
    X = tmp[["Feedback", "Population", "Recording Time","Dataset"]]
    X = pd.get_dummies(X).values
    feats = pf(degree=2, include_bias=False, interaction_only=True)
    X_ = X#feats.fit_transform(X)
    mod = lr().fit(X_, y)
    print(v+" model: R^2 = "+str(mod.score(X_, y)))
#    print(feats.get_feature_names_out(["f","p","r"]))
    print(mod.coef_)
    print(mod.intercept_)